In [1]:
from helpers_given import *
from preprocess import *
from helpers import *

In [2]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("dataset/", sub_sample=False)

y_train loaded
x_train loaded
x_test loaded


In [3]:
y_01 = convert_minus1_to_0(y_train)
nb_col = x_train.shape[1]
nb_rows = x_train.shape[0]
print(nb_col)
print(nb_rows)
#get the features names
with open('dataset/x_train.csv', 'r') as f:
    features_string = f.readline()
    features = features_string.split(',')
features = features[1:]

321
328135


In [4]:
nb_nan = nb_of_nans(x_train)
# Get reduced data
reduced_data, reduced_features, removed_features = removing_features(nb_nan, features, x_train)
# remove the same columns from x_test
reduced_x_test = np.delete(x_test, removed_features, 1)

In [5]:
# Replace nine values with NaNs
replace_nine_with_nan(reduced_data)
replace_nine_with_nan(reduced_x_test)

# Replace seven values with NaNs
replace_seven_with_nan(reduced_data)
replace_seven_with_nan(reduced_x_test)

# Replace 99 values with NaNs
replace_99_with_nan(reduced_data)
replace_99_with_nan(reduced_x_test)

In [6]:
# Remove outliers
reduced_data = clean_outliers_modified(reduced_data)
reduced_test = clean_outliers_modified(reduced_x_test)

# Replace NaNs with medians
reduced_median = replace_NaN(reduced_data, method='median')
reduced_median_test = replace_NaN(reduced_test, method='median')

# Standardize the data
std_x = standardize_data(reduced_median)
std_test = standardize_data(reduced_median_test)

In [7]:
from implementations import *
NB_COL = std_x.shape[1] # corresponds to 'D' = number of features
NB_ROWS = std_x.shape[0] # corresponds to 'N' = number of observations/respondents
print(NB_COL)
print(NB_ROWS)

139
328135


In [10]:
# try with a few columns
mean_0 = np.nanmean(std_x[y_01==0, :], axis=0)
mean_1 = np.nanmean(std_x[y_01==1, :], axis=0)
diff_in_means = np.abs(mean_0 - mean_1)
print(diff_in_means)
print(diff_in_means[diff_in_means>0.5].shape)

[0.0198357  0.02349095 0.01917569 0.02121324 0.08619079 0.04930016
 0.06237066 0.17789656 0.17789656 0.87527377 0.62931321 0.2112384
 0.13877147 0.17210216 0.06834626 0.23499674 0.75378224 0.29656939
 0.72480162 0.17392486 0.29546264 0.29767062 0.65205127 0.58910329
 0.24021757 0.4713792  0.58976609 0.25004332 0.09634421 0.30259886
 0.04542451 0.42181848 0.74436543 0.35625914 0.38322781 0.51982594
 0.08580128 0.05087363 0.64806886 0.67196303 0.36530464 0.34384591
 0.74838728 0.41020246 0.48605093 0.3766478  0.00120159 0.29677999
 0.00373226 0.09704905 0.08070444 0.17022035 0.06212072 0.06830651
 0.27536954 0.21046862 0.10013923 0.24241876 0.61523552 0.14699912
 0.27903938 0.03293938 0.01899329 0.15038133 0.00730785 0.1442304
 0.06709579 0.16837683 0.82627452 0.15916677 0.76382156 0.32491069
 0.17392486 0.19966989 0.19465247 0.58910329 0.07139336 0.04228926
 0.09886118 0.10282697 0.0950762  0.10623404 0.08604412 0.77187475
 0.65288402 0.79140198 0.72910922 0.04270203 0.04208257 0.186343

In [11]:
print(diff_in_means.shape)

(139,)


In [13]:
std_reduced_x = std_x[:, diff_in_means > 0.5]
std_reduced_test = std_x[:, diff_in_means > 0.5]
NB_COL_REDUCED = std_reduced_x.shape[1]
print(NB_COL_REDUCED)

23


In [ ]:
from cross_validation import *

In [27]:
# GD
F1 = np.zeros(5)
for degree in range(5):
    tx = build_poly(std_reduced_x, degree)
    w0 = np.zeros(tx.shape[1])
    w_mse_gd, loss_mse_gd = mean_squared_error_gd(y_01, tx, w0, max_iters = 50, gamma = 0.01)
    y_mse_gd = convert_predict(tx @ w_mse_gd)
    F1[degree] = compute_f1(y_01, y_mse_gd)


c:\ML_Project_1\helpers.py:11: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-z))
c:\ML_Project_1\helpers.py:135: RuntimeWarning: invalid value encountered in scalar divide
  return true_positives / (true_positives + false_positives)


In [28]:
print(F1)

[0.16227493 0.25679431 0.20090015 0.01014631        nan]


In [29]:
# RR
F1_rr = np.zeros(5)
for degree in range(5):
    tx = build_poly(std_reduced_x, degree)
    w0 = np.zeros(tx.shape[1])
    w_mse_gd, loss_mse_gd = ridge_regression(y_01, tx, lambda_=0.5)
    y_mse_gd = convert_predict(tx @ w_mse_gd)
    F1_rr[degree] = compute_f1(y_01, y_mse_gd)
print(F1_rr)

[0.16227493 0.24203733 0.19851635 0.20610448 0.20701987]
